# Flow SDK Real-World Examples

This notebook demonstrates complete, production-ready workflows using Flow SDK. Each example represents a common use case in machine learning and data processing.

## What You'll Learn

1. **End-to-End ML Pipeline** - Data prep → Training → Evaluation → Deployment
2. **Hyperparameter Optimization** - Distributed grid/random search
3. **Large Model Training** - Multi-node distributed training
4. **Data Processing Pipeline** - ETL with checkpointing
5. **ML Development Environment** - Jupyter + experiment tracking
6. **Production Inference Service** - Auto-scaling API deployment

In [ ]:
# Import required modules
from flow import Flow, TaskConfig
import yaml
import json
from datetime import datetime
import os
from pathlib import Path

## 1. End-to-End ML Pipeline

A complete machine learning pipeline from data preparation to model deployment.

In [ ]:
# ML Pipeline configuration
ml_pipeline_yaml = """
# ml-pipeline.yaml
name: ml-pipeline
description: End-to-end ML pipeline for image classification

# Shared configuration
defaults:
  volumes:
    - volume_id: ml-pipeline-data
      mount_path: /data
  environment:
    PROJECT_NAME: image-classifier
    PYTHONPATH: /workspace/src

# Pipeline stages
stages:
  # Stage 1: Data Preparation
  - name: data-preparation
    instance_type: cpu.large
    command: |
      echo "Stage 1: Data Preparation"
      
      # Install dependencies
      pip install pandas numpy pillow scikit-learn
      
      # Download and prepare dataset
      python << 'EOF'
      import os
      import pandas as pd
      from sklearn.model_selection import train_test_split
      
      # Load dataset metadata
      print("Loading dataset...")
      # wget https://example.com/dataset.tar.gz
      # tar -xzf dataset.tar.gz -C /data/raw/
      
      # Create train/val/test splits
      print("Creating data splits...")
      # Process images and create splits
      
      # Save processed data
      print("Saving processed data...")
      # Save to /data/processed/
      
      # Generate data statistics
      stats = {
          "total_samples": 50000,
          "train_samples": 40000,
          "val_samples": 5000,
          "test_samples": 5000,
          "num_classes": 10
      }
      
      import json
      with open('/data/processed/stats.json', 'w') as f:
          json.dump(stats, f)
      
      print("Data preparation complete!")
      print(f"Stats: {stats}")
      EOF

  # Stage 2: Model Training
  - name: model-training
    instance_type: gpu.nvidia.a100
    depends_on: [data-preparation]
    command: |
      echo "Stage 2: Model Training"
      
      # Install ML frameworks
      pip install torch torchvision tensorboard wandb
      
      # Train model
      python << 'EOF'
      import torch
      import torch.nn as nn
      from torch.utils.data import DataLoader
      import json
      import os
      
      # Load data stats
      with open('/data/processed/stats.json') as f:
          stats = json.load(f)
      
      print(f"Training on {stats['train_samples']} samples")
      
      # Define model (ResNet50 for example)
      model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
      model.fc = nn.Linear(2048, stats['num_classes'])
      model = model.cuda()
      
      # Training configuration
      epochs = 10
      batch_size = 64
      learning_rate = 0.001
      
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
      criterion = nn.CrossEntropyLoss()
      
      # Training loop (simplified)
      for epoch in range(epochs):
          print(f"Epoch {epoch+1}/{epochs}")
          # Train on batches
          # ...
          
          # Validation
          val_acc = 0.85 + epoch * 0.01  # Simulated improvement
          print(f"Validation accuracy: {val_acc:.3f}")
          
          # Save checkpoint
          if epoch % 5 == 0:
              checkpoint = {
                  'epoch': epoch,
                  'model_state': model.state_dict(),
                  'optimizer_state': optimizer.state_dict(),
                  'val_acc': val_acc
              }
              torch.save(checkpoint, f'/data/models/checkpoint_epoch_{epoch}.pt')
      
      # Save final model
      torch.save(model.state_dict(), '/data/models/final_model.pt')
      print("Training complete!")
      EOF

  # Stage 3: Model Evaluation
  - name: model-evaluation
    instance_type: gpu.nvidia.t4
    depends_on: [model-training]
    command: |
      echo "Stage 3: Model Evaluation"
      
      pip install torch torchvision matplotlib seaborn
      
      python << 'EOF'
      import torch
      import json
      import numpy as np
      import matplotlib.pyplot as plt
      
      # Load model
      print("Loading trained model...")
      model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=False)
      model.fc = torch.nn.Linear(2048, 10)
      model.load_state_dict(torch.load('/data/models/final_model.pt'))
      model = model.cuda()
      model.eval()
      
      # Evaluate on test set
      print("Evaluating on test set...")
      test_accuracy = 0.92
      test_loss = 0.23
      
      # Generate evaluation report
      report = {
          "test_accuracy": test_accuracy,
          "test_loss": test_loss,
          "precision": 0.91,
          "recall": 0.90,
          "f1_score": 0.905,
          "confusion_matrix": [[95, 5], [8, 92]]  # Simplified 2x2
      }
      
      # Save report
      with open('/data/evaluation/report.json', 'w') as f:
          json.dump(report, f, indent=2)
      
      print(f"Test Accuracy: {test_accuracy:.3f}")
      print("Evaluation complete!")
      EOF

  # Stage 4: Model Deployment Preparation
  - name: deployment-prep
    instance_type: cpu.medium
    depends_on: [model-evaluation]
    command: |
      echo "Stage 4: Deployment Preparation"
      
      # Create deployment package
      python << 'EOF'
      import os
      import json
      import shutil
      import subprocess
      
      # Load evaluation results
      with open('/data/evaluation/report.json') as f:
          report = json.load(f)
      
      # Check if model meets deployment criteria
      if report['test_accuracy'] >= 0.90:
          print("Model approved for deployment!")
          
          # Create deployment package
          os.makedirs('/data/deployment', exist_ok=True)
          
          # Copy model
          shutil.copy('/data/models/final_model.pt', '/data/deployment/model.pt')
          
          # Create model metadata
          metadata = {
              "model_name": "image-classifier-v1",
              "model_type": "resnet50",
              "accuracy": report['test_accuracy'],
              "created_at": str(datetime.now()),
              "framework": "pytorch",
              "input_shape": [3, 224, 224],
              "output_classes": 10
          }
          
          with open('/data/deployment/metadata.json', 'w') as f:
              json.dump(metadata, f, indent=2)
          
          # Create deployment config
          deploy_config = {
              "service_name": "image-classifier-api",
              "model_path": "/models/model.pt",
              "batch_size": 32,
              "workers": 4,
              "gpu_enabled": True
          }
          
          with open('/data/deployment/config.json', 'w') as f:
              json.dump(deploy_config, f, indent=2)
          
          print("Deployment package created!")
      else:
          print(f"Model accuracy {report['test_accuracy']:.3f} below threshold")
          exit(1)
      EOF
"""

# Display pipeline configuration
pipeline_config = yaml.safe_load(ml_pipeline_yaml)
print("ML Pipeline Stages:")
for stage in pipeline_config['stages']:
    print(f"  - {stage['name']}: {stage['instance_type']}")
    if 'depends_on' in stage:
        print(f"    Depends on: {stage['depends_on']}")

In [ ]:
# Execute ML pipeline
def run_ml_pipeline():
    """Execute the complete ML pipeline."""
    with Flow() as flow:
        # Create shared volume
        volume = flow.create_volume(name="ml-pipeline-data", size_gb=100)
        print(f"Created volume: {volume.id}")
        
        # Parse pipeline stages
        pipeline_config = yaml.safe_load(ml_pipeline_yaml)
        defaults = pipeline_config['defaults']
        
        # Track task dependencies
        completed_tasks = {}
        
        # Execute stages in order
        for stage in pipeline_config['stages']:
            # Wait for dependencies
            if 'depends_on' in stage:
                for dep in stage['depends_on']:
                    if dep in completed_tasks:
                        print(f"Waiting for {dep}...")
                        completed_tasks[dep].wait()
            
            # Create task config
            config = TaskConfig(
                name=stage['name'],
                instance_type=stage['instance_type'],
                command=stage['command'],
                volumes=defaults['volumes'],
                environment=defaults['environment']
            )
            
            # Submit task
            print(f"\nStarting stage: {stage['name']}")
            task = flow.run(config, wait=False)
            completed_tasks[stage['name']] = task
        
        # Wait for pipeline completion
        print("\nWaiting for pipeline to complete...")
        for name, task in completed_tasks.items():
            task.wait()
            print(f"{name}: {task.status}")
        
        print("\nPipeline execution complete!")

# Uncomment to run
# run_ml_pipeline()

## 2. Hyperparameter Optimization

Distributed hyperparameter search across multiple configurations.

In [ ]:
# Hyperparameter optimization setup
def create_hyperparam_configs():
    """Generate hyperparameter configurations for grid search."""
    import itertools
    
    # Define search space
    param_grid = {
        'learning_rate': [0.001, 0.01, 0.1],
        'batch_size': [32, 64, 128],
        'optimizer': ['adam', 'sgd'],
        'dropout': [0.2, 0.5],
        'hidden_size': [128, 256, 512]
    }
    
    # Generate all combinations
    keys = param_grid.keys()
    values = param_grid.values()
    
    configs = []
    for combination in itertools.product(*values):
        config = dict(zip(keys, combination))
        configs.append(config)
    
    return configs

# Generate configurations
hyperparam_configs = create_hyperparam_configs()
print(f"Total configurations: {len(hyperparam_configs)}")
print("\nFirst 5 configurations:")
for i, config in enumerate(hyperparam_configs[:5]):
    print(f"{i+1}: {config}")

In [ ]:
# Hyperparameter optimization task
hyperparam_task_template = """
#!/usr/bin/env python
import os
import json
import time
import torch
import torch.nn as nn
import numpy as np

# Load hyperparameters
params = {params_json}
print(f"Training with parameters: {params}")

# Define model based on hyperparameters
class SimpleModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, output_size=10, dropout=0.5):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize model
model = SimpleModel(
    hidden_size=params['hidden_size'],
    dropout=params['dropout']
).cuda()

# Setup optimizer
if params['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=params['learning_rate'])

# Training loop (simplified)
epochs = 20
best_val_acc = 0

for epoch in range(epochs):
    # Simulate training
    train_loss = 1.0 / (epoch + 1) + np.random.normal(0, 0.1)
    
    # Simulate validation
    val_acc = min(0.95, 0.7 + epoch * 0.01 + params['learning_rate'] * 2)
    val_acc += np.random.normal(0, 0.02)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
    
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_acc={val_acc:.4f}")
    time.sleep(0.5)  # Simulate work

# Save results
results = {
    'params': params,
    'best_val_acc': float(best_val_acc),
    'final_train_loss': float(train_loss),
    'task_id': os.environ.get('FLOW_TASK_ID', 'unknown')
}

# Save to shared volume
result_file = f"/results/hyperparam_{params['learning_rate']}_{params['batch_size']}_{params['hidden_size']}.json"
os.makedirs(os.path.dirname(result_file), exist_ok=True)
with open(result_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nBest validation accuracy: {best_val_acc:.4f}")
print(f"Results saved to: {result_file}")
"""

# Launch hyperparameter search
def run_hyperparam_search(configs, max_parallel=10):
    """Run distributed hyperparameter search."""
    with Flow() as flow:
        # Create shared results volume
        results_volume = flow.create_volume(name="hyperparam-results", size_gb=10)
        
        tasks = []
        
        # Submit tasks in batches
        for i, params in enumerate(configs[:max_parallel]):
            # Create task-specific script
            script = hyperparam_task_template.format(params_json=json.dumps(params))
            
            config = TaskConfig(
                name=f"hyperparam-{i}",
                instance_type="gpu.nvidia.t4",
                command=f"python -c '{script}'",
                volumes=[{
                    "volume_id": results_volume.id,
                    "mount_path": "/results"
                }]
            )
            
            task = flow.run(config, wait=False)
            tasks.append((params, task))
            print(f"Submitted task {i+1}/{len(configs)}")
        
        # Wait for completion and collect results
        print("\nWaiting for tasks to complete...")
        results = []
        
        for params, task in tasks:
            task.wait()
            if task.status.value == "completed":
                results.append(params)
        
        print(f"\nCompleted {len(results)} experiments")
        return results

# Example usage
print("Hyperparameter search configuration:")
print(f"  Total experiments: {len(hyperparam_configs)}")
print(f"  Instance type: gpu.nvidia.t4")
print(f"  Parallel tasks: 10")
print("\nTo run: results = run_hyperparam_search(hyperparam_configs)")

## 3. Large Model Training - Multi-Node Distributed

Training large language models across multiple GPUs and nodes.

In [ ]:
# Large model distributed training configuration
large_model_config = TaskConfig(
    name="llm-training",
    instance_type="gpu.nvidia.a100",  # 8x A100 GPUs per node
    instance_count=4,  # 4 nodes = 32 GPUs total
    environment={
        "MODEL_NAME": "gpt-7b",
        "BATCH_SIZE": "4",  # Per GPU
        "GRADIENT_ACCUMULATION_STEPS": "8",
        "LEARNING_RATE": "1e-4",
        "WARMUP_STEPS": "2000",
        "SAVE_STEPS": "1000",
        "WANDB_PROJECT": "llm-training"
    },
    volumes=[
        {"volume_id": "training-data-vol", "mount_path": "/data"},
        {"volume_id": "model-checkpoints-vol", "mount_path": "/checkpoints"}
    ],
    command="""
#!/bin/bash
set -e

echo "=== Large Model Distributed Training ==="
echo "Node: $FLOW_NODE_RANK / $FLOW_NODE_COUNT"
echo "Master: $FLOW_NODE_0_IP"

# Install dependencies
pip install torch transformers datasets accelerate deepspeed wandb

# Setup distributed environment
export MASTER_ADDR=$FLOW_NODE_0_IP
export MASTER_PORT=29500
export WORLD_SIZE=$((FLOW_NODE_COUNT * 8))  # 8 GPUs per node
export RANK=$((FLOW_NODE_RANK * 8))

# Create DeepSpeed configuration
cat > ds_config.json << 'DEEPSPEED_EOF'
{
  "train_batch_size": 128,
  "gradient_accumulation_steps": 8,
  "fp16": {
    "enabled": true
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu"
    },
    "offload_param": {
      "device": "cpu"
    },
    "overlap_comm": true,
    "contiguous_gradients": true,
    "reduce_bucket_size": 1e8
  },
  "gradient_clipping": 1.0,
  "wall_clock_breakdown": true
}
DEEPSPEED_EOF

# Training script
python << 'TRAINING_EOF'
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
import wandb

# Initialize W&B on rank 0
if int(os.environ.get('RANK', 0)) == 0:
    wandb.init(
        project=os.environ.get('WANDB_PROJECT', 'llm-training'),
        name=f"{os.environ['MODEL_NAME']}-{os.environ['FLOW_TASK_ID']}"
    )

print(f"Loading model: {os.environ['MODEL_NAME']}")

# Load model and tokenizer
model_name = "gpt2"  # Use smaller model for demo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token
tokenizer.pad_token = tokenizer.eos_token

# Load dataset
print("Loading dataset...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Setup training arguments
training_args = TrainingArguments(
    output_dir="/checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=int(os.environ['BATCH_SIZE']),
    per_device_eval_batch_size=int(os.environ['BATCH_SIZE']),
    gradient_accumulation_steps=int(os.environ['GRADIENT_ACCUMULATION_STEPS']),
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=int(os.environ['SAVE_STEPS']),
    warmup_steps=int(os.environ['WARMUP_STEPS']),
    learning_rate=float(os.environ['LEARNING_RATE']),
    logging_steps=10,
    logging_dir="/checkpoints/logs",
    deepspeed="ds_config.json",
    fp16=True,
    report_to=["wandb"] if int(os.environ.get('RANK', 0)) == 0 else [],
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Start training
print("Starting distributed training...")
trainer.train()

# Save final model
if int(os.environ.get('RANK', 0)) == 0:
    print("Saving final model...")
    trainer.save_model("/checkpoints/final")
    tokenizer.save_pretrained("/checkpoints/final")

print("Training complete!")
TRAINING_EOF
    """
)

print("Large Model Training Configuration:")
print(f"  Model: {large_model_config.environment['MODEL_NAME']}")
print(f"  Nodes: {large_model_config.instance_count}")
print(f"  Total GPUs: {large_model_config.instance_count * 8}")
print(f"  Global batch size: {32 * 4 * 8}")
print("  Optimization: DeepSpeed ZeRO-3")

## 4. Data Processing Pipeline with Checkpointing

Large-scale data processing with fault tolerance and checkpointing.

In [ ]:
# Data processing pipeline with checkpointing
data_pipeline_script = """
#!/usr/bin/env python
import os
import json
import time
import pickle
from datetime import datetime
import hashlib

class CheckpointedProcessor:
    def __init__(self, checkpoint_dir="/data/checkpoints"):
        self.checkpoint_dir = checkpoint_dir
        os.makedirs(checkpoint_dir, exist_ok=True)
        self.checkpoint_file = os.path.join(checkpoint_dir, "processor_state.pkl")
        
    def load_checkpoint(self):
        """Load processing state from checkpoint."""
        if os.path.exists(self.checkpoint_file):
            with open(self.checkpoint_file, 'rb') as f:
                state = pickle.load(f)
                print(f"Resuming from checkpoint: {state['processed_count']} files processed")
                return state
        return {
            'processed_files': set(),
            'processed_count': 0,
            'failed_files': [],
            'start_time': datetime.now().isoformat()
        }
    
    def save_checkpoint(self, state):
        """Save current processing state."""
        with open(self.checkpoint_file, 'wb') as f:
            pickle.dump(state, f)
    
    def process_file(self, filepath):
        """Process a single file with error handling."""
        try:
            # Simulate file processing
            print(f"Processing: {filepath}")
            
            # Read file (simulated)
            file_size = os.path.getsize(filepath) if os.path.exists(filepath) else 1000000
            
            # Simulate processing time based on file size
            process_time = min(file_size / 1000000, 5)  # Max 5 seconds
            time.sleep(process_time)
            
            # Generate output
            output = {
                'input_file': filepath,
                'file_size': file_size,
                'processed_at': datetime.now().isoformat(),
                'checksum': hashlib.md5(filepath.encode()).hexdigest(),
                'records_processed': file_size // 100
            }
            
            # Save output
            output_file = filepath.replace('/input/', '/output/').replace('.raw', '.json')
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, 'w') as f:
                json.dump(output, f)
            
            return True, output
            
        except Exception as e:
            print(f"Error processing {filepath}: {e}")
            return False, str(e)
    
    def run(self, input_files):
        """Run processing pipeline with checkpointing."""
        # Load checkpoint
        state = self.load_checkpoint()
        
        # Process files
        for filepath in input_files:
            # Skip already processed files
            if filepath in state['processed_files']:
                continue
            
            # Process file
            success, result = self.process_file(filepath)
            
            if success:
                state['processed_files'].add(filepath)
                state['processed_count'] += 1
            else:
                state['failed_files'].append({
                    'file': filepath,
                    'error': result,
                    'timestamp': datetime.now().isoformat()
                })
            
            # Save checkpoint every 10 files
            if state['processed_count'] % 10 == 0:
                self.save_checkpoint(state)
                print(f"Checkpoint saved: {state['processed_count']} files processed")
        
        # Final checkpoint
        state['end_time'] = datetime.now().isoformat()
        self.save_checkpoint(state)
        
        # Generate summary
        summary = {
            'total_files': len(input_files),
            'processed_files': state['processed_count'],
            'failed_files': len(state['failed_files']),
            'start_time': state['start_time'],
            'end_time': state['end_time']
        }
        
        with open('/data/processing_summary.json', 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f"\nProcessing complete!")
        print(f"Processed: {summary['processed_files']}")
        print(f"Failed: {summary['failed_files']}")

# Main execution
if __name__ == "__main__":
    # Get list of input files
    input_files = []
    for i in range(100):
        input_files.append(f"/data/input/file_{i:04d}.raw")
    
    # Run processor
    processor = CheckpointedProcessor()
    processor.run(input_files)
"""

# Create data processing task
data_processing_config = TaskConfig(
    name="data-processing-pipeline",
    instance_type="cpu.large",
    instance_count=5,  # Process in parallel
    volumes=[
        {"volume_id": "raw-data-vol", "mount_path": "/data/input"},
        {"volume_id": "processed-data-vol", "mount_path": "/data/output"},
        {"name": "checkpoints", "size_gb": 10, "mount_path": "/data/checkpoints"}
    ],
    command=f"python -c '{data_pipeline_script}'"
)

print("Data Processing Pipeline:")
print("  Features:")
print("    - Automatic checkpointing")
print("    - Failure recovery")
print("    - Parallel processing")
print("    - Progress tracking")
print(f"  Workers: {data_processing_config.instance_count}")
print(f"  Instance type: {data_processing_config.instance_type}")

## 5. ML Development Environment

Complete ML development environment with Jupyter, experiment tracking, and model serving.

In [ ]:
# ML Development Environment
ml_dev_env_config = TaskConfig(
    name="ml-dev-environment",
    instance_type="gpu.nvidia.a10g",
    ports=[8888, 6006, 8501, 5000, 8000],  # Jupyter, TensorBoard, Streamlit, API, Prometheus
    volumes=[
        {"name": "workspace", "size_gb": 200, "mount_path": "/workspace"},
        {"name": "datasets", "size_gb": 500, "mount_path": "/datasets"},
        {"name": "models", "size_gb": 100, "mount_path": "/models"}
    ],
    environment={
        "JUPYTER_TOKEN": "ml-dev-token-123",
        "MLFLOW_TRACKING_URI": "http://localhost:5000"
    },
    command="""
#!/bin/bash
set -e

echo "=== Setting up ML Development Environment ==="

# Install comprehensive ML stack
pip install --upgrade pip
pip install \
    jupyter jupyterlab ipywidgets \
    numpy pandas scikit-learn matplotlib seaborn plotly \
    torch torchvision torchaudio transformers datasets \
    tensorflow tensorboard \
    mlflow wandb \
    streamlit gradio \
    fastapi uvicorn \
    pytest black flake8 mypy

# Install Jupyter extensions
jupyter nbextension enable --py widgetsnbextension

# Setup workspace structure
mkdir -p /workspace/{notebooks,src,experiments,configs}

# Create welcome notebook
cat > /workspace/notebooks/Welcome.ipynb << 'NOTEBOOK_EOF'
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ML Development Environment\\n",
    "\\n",
    "## Available Services\\n",
    "- **Jupyter Lab**: http://localhost:8888 (token: ml-dev-token-123)\\n",
    "- **TensorBoard**: http://localhost:6006\\n",
    "- **MLflow**: http://localhost:5000\\n",
    "- **API Server**: http://localhost:8000\\n",
    "\\n",
    "## Quick Start\\n",
    "1. Check GPU availability\\n",
    "2. Load a dataset\\n",
    "3. Train a model\\n",
    "4. Track experiments"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\\n",
    "import tensorflow as tf\\n",
    "print(f'PyTorch: {torch.__version__}')\\n",
    "print(f'TensorFlow: {tf.__version__}')\\n",
    "print(f'CUDA Available: {torch.cuda.is_available()}')\\n",
    "if torch.cuda.is_available():\\n",
    "    print(f'GPU: {torch.cuda.get_device_name(0)}')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
NOTEBOOK_EOF

# Start MLflow server
echo "Starting MLflow server..."
mlflow server \
    --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:////workspace/mlflow.db \
    --default-artifact-root /workspace/mlflow-artifacts &

# Start TensorBoard
echo "Starting TensorBoard..."
tensorboard --logdir=/workspace/experiments --port=6006 --bind_all &

# Create simple model serving API
cat > /workspace/api.py << 'API_EOF'
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import numpy as np

app = FastAPI(title="ML Model API")

class PredictionRequest(BaseModel):
    data: list

class PredictionResponse(BaseModel):
    prediction: list
    confidence: float

@app.get("/")
def root():
    return {"message": "ML Model API", "status": "ready"}

@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    # Dummy prediction
    data = np.array(request.data)
    prediction = data.mean(axis=0).tolist()
    confidence = np.random.uniform(0.8, 0.99)
    return PredictionResponse(prediction=prediction, confidence=confidence)

@app.get("/health")
def health():
    return {"status": "healthy", "gpu_available": torch.cuda.is_available()}
API_EOF

# Start API server
echo "Starting API server..."
cd /workspace && uvicorn api:app --host 0.0.0.0 --port 8000 &

# Start Jupyter Lab (foreground)
echo "Starting Jupyter Lab..."
jupyter lab \
    --ip=0.0.0.0 \
    --port=8888 \
    --no-browser \
    --allow-root \
    --NotebookApp.token=$JUPYTER_TOKEN \
    --NotebookApp.notebook_dir=/workspace
    """
)

print("ML Development Environment:")
print("  Services:")
print("    - JupyterLab (port 8888)")
print("    - MLflow Tracking (port 5000)")
print("    - TensorBoard (port 6006)")
print("    - Model API (port 8000)")
print("  Storage:")
print("    - Workspace: 200GB")
print("    - Datasets: 500GB")
print("    - Models: 100GB")

## 6. Production Inference Service

Auto-scaling production inference service with monitoring.

In [ ]:
# Production inference service
inference_service_yaml = """
name: model-inference-service
instance_type: gpu.nvidia.t4
min_instances: 2
max_instances: 10
scale_up_threshold: 0.8  # CPU/GPU utilization
scale_down_threshold: 0.3

ports:
  - 8080  # API endpoint
  - 9090  # Prometheus metrics

health_check:
  endpoint: /health
  interval: 30
  timeout: 10
  unhealthy_threshold: 3

volumes:
  - volume_id: production-models
    mount_path: /models

environment:
  MODEL_NAME: image-classifier-v2
  MODEL_PATH: /models/image-classifier-v2.pt
  BATCH_SIZE: 32
  MAX_BATCH_WAIT_MS: 100
  WORKERS: 4

command: |
  #!/bin/bash
  
  # Install dependencies
  pip install fastapi uvicorn torch torchvision pillow prometheus-client
  
  # Create inference service
  cat > inference_service.py << 'SERVICE_EOF'
  import os
  import time
  import torch
  import torchvision.transforms as transforms
  from fastapi import FastAPI, File, UploadFile, HTTPException
  from fastapi.responses import JSONResponse
  from prometheus_client import Counter, Histogram, Gauge, generate_latest
  from PIL import Image
  import io
  import asyncio
  from typing import List
  import numpy as np
  
  # Metrics
  request_count = Counter('inference_requests_total', 'Total inference requests')
  request_duration = Histogram('inference_duration_seconds', 'Inference duration')
  batch_size_gauge = Gauge('inference_batch_size', 'Current batch size')
  gpu_utilization = Gauge('gpu_utilization_percent', 'GPU utilization')
  
  app = FastAPI(title="Model Inference Service")
  
  # Load model
  print(f"Loading model from {os.environ['MODEL_PATH']}")
  model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
  model.eval()
  if torch.cuda.is_available():
      model = model.cuda()
  
  # Image preprocessing
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  
  # Batch collector
  class BatchCollector:
      def __init__(self, max_batch_size=32, max_wait_ms=100):
          self.max_batch_size = max_batch_size
          self.max_wait_ms = max_wait_ms
          self.batch = []
          self.futures = []
          self.lock = asyncio.Lock()
          self.last_batch_time = time.time()
  
  batch_collector = BatchCollector(
      max_batch_size=int(os.environ['BATCH_SIZE']),
      max_wait_ms=int(os.environ['MAX_BATCH_WAIT_MS'])
  )
  
  @app.post("/predict")
  async def predict(file: UploadFile = File(...)):
      request_count.inc()
      
      with request_duration.time():
          # Read and preprocess image
          contents = await file.read()
          image = Image.open(io.BytesIO(contents))
          input_tensor = preprocess(image)
          
          # Add to batch
          future = asyncio.Future()
          async with batch_collector.lock:
              batch_collector.batch.append(input_tensor)
              batch_collector.futures.append(future)
              
              # Process batch if full or timeout
              if len(batch_collector.batch) >= batch_collector.max_batch_size or \
                 (time.time() - batch_collector.last_batch_time) * 1000 > batch_collector.max_wait_ms:
                  await process_batch()
          
          # Wait for result
          result = await future
          return JSONResponse(result)
  
  async def process_batch():
      """Process accumulated batch."""
      batch = torch.stack(batch_collector.batch)
      futures = batch_collector.futures
      
      batch_size_gauge.set(len(batch))
      
      # Clear batch
      batch_collector.batch = []
      batch_collector.futures = []
      batch_collector.last_batch_time = time.time()
      
      # Run inference
      if torch.cuda.is_available():
          batch = batch.cuda()
      
      with torch.no_grad():
          outputs = model(batch)
          probabilities = torch.nn.functional.softmax(outputs, dim=1)
          predictions = probabilities.argmax(dim=1)
      
      # Return results
      for i, future in enumerate(futures):
          result = {
              "class": int(predictions[i]),
              "confidence": float(probabilities[i].max()),
              "top_5": probabilities[i].topk(5).indices.tolist()
          }
          future.set_result(result)
  
  @app.get("/health")
  def health_check():
      return {
          "status": "healthy",
          "model": os.environ['MODEL_NAME'],
          "gpu_available": torch.cuda.is_available()
      }
  
  @app.get("/metrics")
  def metrics():
      return generate_latest()
  
  # Background GPU monitoring
  async def monitor_gpu():
      while True:
          if torch.cuda.is_available():
              # Get GPU utilization (simulated)
              util = torch.cuda.utilization()
              gpu_utilization.set(util)
          await asyncio.sleep(10)
  
  @app.on_event("startup")
  async def startup_event():
      asyncio.create_task(monitor_gpu())
  
  SERVICE_EOF
  
  # Start service
  uvicorn inference_service:app --host 0.0.0.0 --port 8080 --workers $WORKERS
"""

print("Production Inference Service:")
print("  Features:")
print("    - Auto-scaling (2-10 instances)")
print("    - Batch inference")
print("    - Health checks")
print("    - Prometheus metrics")
print("    - GPU optimization")
print("  Performance:")
print("    - Batch size: 32")
print("    - Max latency: 100ms")
print("    - Workers: 4")

## Summary

This notebook demonstrated production-ready workflows using Flow SDK:

### 1. **End-to-End ML Pipeline**
- Multi-stage pipeline with dependencies
- Data prep → Training → Evaluation → Deployment
- Automatic checkpoint and model versioning

### 2. **Hyperparameter Optimization**
- Distributed grid/random search
- Parallel experiment execution
- Result aggregation and analysis

### 3. **Large Model Training**
- Multi-node distributed training
- DeepSpeed optimization
- Gradient accumulation and mixed precision

### 4. **Data Processing Pipeline**
- Fault-tolerant processing with checkpoints
- Parallel data transformation
- Progress tracking and recovery

### 5. **ML Development Environment**
- Complete ML stack (Jupyter, MLflow, TensorBoard)
- Persistent workspace
- Integrated tooling

### 6. **Production Inference**
- Auto-scaling service
- Batch inference optimization
- Monitoring and health checks

### Best Practices Applied

- **Cost Optimization**: Use appropriate instance types, set runtime limits
- **Fault Tolerance**: Checkpointing, error handling, retries
- **Monitoring**: Metrics, logging, health checks
- **Scalability**: Multi-node support, auto-scaling
- **Reproducibility**: Version control, experiment tracking

### Next Steps

1. Adapt these examples to your specific use cases
2. Integrate with your existing ML infrastructure
3. Explore Flow SDK's advanced features
4. Join the community for support and best practices

Happy building with Flow SDK! 🚀